In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

def yf_extract_info(item):
    link=item.find('a',class_='subtle-link')['href']
    title=item.find('a',class_='subtle-link')['title']

    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Run in headless mode
    chrome_options.add_argument("--disable-gpu")  # Disable GPU acceleration

    page_driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    page_driver.get(link)

    page_soup = BeautifulSoup(page_driver.page_source, 'html.parser')
    content = page_soup.find('div', class_='body')
    content_text = content.text.strip() if content else ''
    date= page_soup.find('div', class_= 'byline').find('time')
    data_point = {'Date': date.text.strip() if date else 'N/A', 'Content': 'Title:' + title + '\nArticle:' + content_text}
    page_driver.quit()
    return data_point

def get_yf_articles(URL):

    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Run in headless mode
    chrome_options.add_argument("--disable-gpu")  # Disable GPU acceleration
    
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    driver.get(URL)

    time.sleep(2)

    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # Wait for new content to load
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    articles = soup.find_all('li', class_='story-item')
    news_data = []
    for article in articles:
        try:
            news_data.append(yf_extract_info(article))
        except Exception as e:
            print(f"Error extracting article: {e}")
            print(article)
            continue
    driver.quit()
    return pd.DataFrame(news_data)

URL="https://finance.yahoo.com/news/"
yf_df=get_yf_articles(URL)
yf_df['Date'] = pd.to_datetime(yf_df['Date'], errors='coerce')
yf_df.to_csv('yahoo_finance_articles.csv', index=False)

Error extracting article: 'NoneType' object has no attribute 'find'
<li class="stream-item story-item yf-1drgw5l"><section class="container sz-xx-large optimizedStream yf-1jsv3x8 responsive rev showMobileThumbnails" data-testid="storyitem" role="article"><a aria-label="Analyst Report: RH" class="subtle-link fin-size-small thumb yf-17w8w6" data-rapid_p="21" data-ylk="elm:img;elmt:link;itc:0;ct:story;slk:Analyst%20Report%3A%20RH;sec:news-stream;subsec:fltrd-strs;cpos:7;g:4c955458-f08f-3af3-9ea4-9953826eafa5" href="https://finance.yahoo.com/research/reports/MS_0P0000X96M_AnalystReport_1749791025000?yptr=yahoo&amp;ncid=yahooproperties_plusresear_nm5q6ze1cei" title="Analyst Report: RH"><div style="display: contents; --border-radius:var(--border-radius-m); --aspect-ratio:16/9;"> <img alt="Analyst Report: RH" class="tw-bg-opacity-25 yf-13q9uv1" fetchpriority="auto" loading="lazy" sizes="12.125rem" src="https://s.yimg.com/uu/api/res/1.2/T0vKR8.UUSwF3s8pOO5fnA--~B/aD0wO3c9MDthcHBpZD15dGFjaHlvbg

C:\Users\dsygn\AppData\Local\Temp\ipykernel_57212\4242752305.py:64: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  yf_df['Date'] = pd.to_datetime(yf_df['Date'], errors='coerce')
C:\Users\dsygn\AppData\Local\Temp\ipykernel_57212\4242752305.py:64: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  yf_df['Date'] = pd.to_datetime(yf_df['Date'], errors='coerce')
